# Zadanie nr 6 - Wzorce dwuwymiarowe

In [26]:
import networkx as nx
import matplotlib.pyplot as plt
from time import time

## 0. Funkcje pomocnicze

In [16]:
def print2d(l): 
    print('\n'.join(map(''.join, list(map(lambda x: str(x).replace("'", ""), l)))), '\n')

In [17]:
def show_graph(head):
    graph = nx.DiGraph()
    stack = [head]

    while stack:
        node = stack.pop()
        for key, edge_node in node.edges.items():
            graph.add_edge(node, edge_node, label=key)

            if not edge_node.visited:
                stack.append(edge_node)

        if node.fail_link:
            graph.add_edge(node, node.fail_link)

        node.visited = True

    pos = nx.spectral_layout(graph)
    nx.draw_networkx(
        graph, pos, connectionstyle='arc3, rad = 0.2', node_color='turquoise')
    labels = nx.get_edge_attributes(graph, 'label')
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels)
    plt.show()

In [18]:
def kmp(text, pattern):
    result = []

    pi = prefix_function(pattern)
    q = 0

    for i in range(0, len(text)):
        while q > 0 and pattern[q] != text[i]:
            q = pi[q-1]

        if pattern[q] == text[i]:
            q = q + 1

        if q == len(pattern):
            result.append(i + 1 - q)
            q = pi[q-1]

    return result


def prefix_function(pattern):
    pi = [0]
    k = 0

    for q in range(1, len(pattern)):
        while(k > 0 and pattern[k] != pattern[q]):
            k = pi[k-1]

        if(pattern[k] == pattern[q]):
            k = k + 1

        pi.append(k)

    return pi

## 1. Algorytm

<i>1. Zaimplementuj algorytm wyszukiwania wzorca 2-wymiarowego </i>

In [19]:
class Node:
    index = 0

    def __init__(self, parent=None, char=None):
        self.edges = {}
        self.fail_link = None
        self.parent = parent
        self.char = char
        self.visited = False
        self.id = Node.index
        Node.index += 1
        self.pattern_end = None

    def __lt__(self, other):
        return self.char < other.char

    def __str__(self):
        return str(self.id)

In [20]:
def build_trie(patterns):
    head = Node()

    for i, pattern in enumerate(patterns):
        current_node = head
        for char in pattern:
            if char in current_node.edges:
                current_node = current_node.edges[char]
            else:
                current_node.edges[char] = Node(char=char, parent=current_node)
                current_node = current_node.edges[char]

        current_node.pattern_end = i+1

    return head

In [21]:
def build_automaton(patterns):
    head = build_trie(patterns)
    # show_graph(head)

    queue = [head]
    visited = []

    def assign_fail_links(node):
        if node is not head:
            current_node = node.parent.fail_link
            while current_node:
                if node.char in current_node.edges:
                    node.fail_link = current_node.edges[node.char]
                    break
                current_node = current_node.fail_link
            else:
                node.fail_link = head

        for edge in node.edges.values():
            if edge not in visited:
                queue.append(edge)

        visited.append(node)

    while queue:
        node = queue.pop(0)
        assign_fail_links(node)

    return head

In [22]:
def modify_automaton(head):

    queue = [head]
    visited = []

    def copy_edges(node):
        if node is not head:
            for key, node_edge in node.fail_link.edges.items():
                if key not in node.edges:
                    node.edges[key] = node_edge

        for edge in node.edges.values():
            if edge not in visited and edge not in queue:
                queue.append(edge)
                visited.append(edge)

        visited.append(node)
        node.fail_link = None

    while queue:
        node = queue.pop(0)
        copy_edges(node)

    return head

In [23]:
def pattern_match_2d(text, pattern):
    head = build_automaton(pattern)
    # show_graph(head)
    head = modify_automaton(head)
    # show_graph(head)

    patterns_matrix = [[0 for _ in range(len(text[0]))]
                       for _ in range(len(text))]

    for i, line in enumerate(text):
        current_node = head
        for j, char in enumerate(line):
            if char in current_node.edges:
                current_node = current_node.edges[char]
            else:
                current_node = head

            if current_node.pattern_end:
                patterns_matrix[i][j] = current_node.pattern_end

    print2d(patterns_matrix)

    col_pattern = [i+1 for i in range(len(pattern[0]))]

    result = []

    for i in range(len(text[0])):
        line = [patterns_matrix[j][i] for j in range(len(text))]
        print(line, col_pattern)
        for j in kmp(line, col_pattern):
            result.append((j, i-len(pattern[0])+1))

    print2d(result)
    return result

In [25]:
text = [[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 9, 9]]
pattern = [[1, 2], [4, 5]]

pattern_match_2d(text, pattern)

[0, 0, 1]
[0, 0, 2]
[0, 0, 0]
[0, 0, 0] 

[0, 0, 0, 0] [1, 2]
[0, 0, 0, 0] [1, 2]
[1, 2, 0, 0] [1, 2]
(0, 1) 



[(0, 1)]

## 2. Testy

<i>2. Znajdź w załączonym pliku "haystack.txt" wszyskie sytuacje, gdy taka sama litera występuje na tej samej pozycji w dwóch kolejnych linijkach. Zwróć uwagę, na nierówną długość linii w pliku.</i>

<i>3. Znajdź wszystkie wystąpienia "th" oraz "t h" w dwóch kolejnych liniach na tej samej pozycji.</i>

<i>4. Wybierz przynajmniej 3 litery (małe). Znajdź wszystkie wystąpienia tej litery w załączonym pliku "haystack.png"</i>

<i>5. Znajdź wszystkie wystąpienia słowa "p a t t e r n" w haystack.png.</i>

## 3. Porównanie czasów

<i>6. Porównaj czas budowania automatu i czas wyszukiwania dla różnych rozmiarów wzorca</i>

<i>7. Podziel plik na 2, 4 i 8 fragmentów (w poziomie) i porównaj czas przeszukiwania</i>

## Wnioski

- 
- 

M. Hawryluk 03.06.2021